# ブログからコメントデータを簡単に取得する例

## 指定したブログの各エントリのコメント欄から，テキストデータを抽出する

**仕組み**

1. ブログコメントのソースURLを指定する．
2. ブログコメント欄の，コメントテキストの先頭と終端の目印になるHTMLタグをソースコードから調査して，`start_comment`, `end_comment`変数に代入
3. 両変数に挟まれた文字列をコメントテキストとして抽出．抽出したコメントは`comment_list`に追加される
4. 3をコメント数分だけ実行
5. 一秒間のsleepを挟む

1-4を各エントリに対して実行する．一部エントリは消されているものもあるので，404エラーが出現したときは，次のエントリにパスするように例外処理する．

### e.g. xxxx.comというブログコメントを収集する場合

In [ ]:
import numpy as np
import time
import sys
import urllib
comment_list=[]

# 参照するエントリの範囲を指定(7509エントリまで存在している場合)
article_idxes = np.arange(1, 7509)

for article_idx in article_idxes:
    # 各エントリのURL
    src_url = "http://xxxx.com/blog-entry-" + str(article_idx) + ".html"
    # コメントテキストの先頭
    start_comment = "<strong></strong></p>" 
    len_start = len(start_comment)
    # コメントテキストの終端
    end_comment = "<div class=\"comment_navi\">"
    len_end = len(end_comment)

    try:
        response = urllib.request.urlopen(src_url)
        content_b = response.read()
        content_u = content_b.decode("utf-8")
    except HTTPError:
        print("エントリ{:d}のコンテンツ取得時にHTTPError(404)発生".format(article_idx))
        pass
    except :
        print("エントリ{:d}のコンテンツ取得時にHTTPError以外のエラー発生".format(article_idx))
        pass
    else:
        offset = 0
        is_terminated = 0
        while is_terminated !=1 :
            content_tmp = content_u[offset:]
            start_head_idx = content_tmp.find("<strong></strong></p>")
            end_head_idx = content_tmp.find("<div class=\"comment_navi\">")
            if start_head_idx != -1:
                comment = content_tmp[start_head_idx+len_start:end_head_idx].lstrip("\r\n")
                comment_list.append(comment)
                # update offset for the next comment searching
                offset += end_head_idx + len_end
            else:
                is_terminated = 1
        sys.stdout.write("\r エントリ{:d}のコメント抽出が完了".format(article_idx))
        sys.stdout.flush()
        
    time.sleep(1)

## 収集したコメントをcsvファイルとして保存する

In [ ]:
import pandas as pd
train_sample_df = pd.DataFrame(comment_list, columns=["comment_text"])
# ラベルを仮にすべて0として指定
is_toxic = np.zeros(len(comment_list), dtype="uint8")
train_sample_df["is_toxic"] = is_toxic

In [ ]:
train_sample_df.to_csv("train_sample.csv", index=False)